In [22]:
print("hello this is inside the jupyter notebook")

hello this is inside the jupyter notebook


In [23]:
print("Hello, M&A AnalystGPT!")
import langchain
print(langchain.__version__)

Hello, M&A AnalystGPT!
0.3.25


In [24]:
import fitz
doc = fitz.open("amazon_10k_2024.pdf")  # Adjust filename as needed
text = "".join(page.get_text() for page in doc)
chunks = [text[i:i+500] for i in range(0, len(text), 500)]
print(f"Created {len(chunks)} chunks")

Created 591 chunks


In [25]:
import fitz

pdf_files = [
    "amazon_10k_2024.pdf",
    "Tesla_10k_2024.pdf",
    "ExxonMobil_10k_2024.pdf",
    "apple_10k_2024.pdf",
    "Microsoft_10k_2024.pdf",
    "Alphabet_10k_2024.pdf",
    "JPMorgan_10k_2024.pdf",
    "Johnson&Johnson_10k_2024.pdf",
    "walmart_10k_2024.pdf",
    "Pepsico_10k_2024.pdf"
]

all_chunks = []
for pdf_file in pdf_files:
    print(f"Processing {pdf_file}...")
    try:
        doc = fitz.open(pdf_file)
        text = "".join(page.get_text() for page in doc)
        chunks = [text[i:i+500] for i in range(0, len(text), 500)]
        all_chunks.extend(chunks)
        print(f"Added {len(chunks)} chunks from {pdf_file}")
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")

print(f"Total chunks created: {len(all_chunks)}")
with open("all_chunks.txt", "w", encoding="utf-8") as f:
    for i, chunk in enumerate(all_chunks, 1):
        f.write(f"Chunk {i}: {chunk}\n")
print("Chunks saved to all_chunks.txt")

Processing amazon_10k_2024.pdf...
Added 591 chunks from amazon_10k_2024.pdf
Processing Tesla_10k_2024.pdf...
Added 812 chunks from Tesla_10k_2024.pdf
Processing ExxonMobil_10k_2024.pdf...
Added 946 chunks from ExxonMobil_10k_2024.pdf
Processing apple_10k_2024.pdf...
Added 429 chunks from apple_10k_2024.pdf
Processing Microsoft_10k_2024.pdf...
Added 807 chunks from Microsoft_10k_2024.pdf
Processing Alphabet_10k_2024.pdf...
Added 742 chunks from Alphabet_10k_2024.pdf
Processing JPMorgan_10k_2024.pdf...
Added 2515 chunks from JPMorgan_10k_2024.pdf
Processing Johnson&Johnson_10k_2024.pdf...
Added 874 chunks from Johnson&Johnson_10k_2024.pdf
Processing walmart_10k_2024.pdf...
Added 775 chunks from walmart_10k_2024.pdf
Processing Pepsico_10k_2024.pdf...
Added 926 chunks from Pepsico_10k_2024.pdf
Total chunks created: 9417
Chunks saved to all_chunks.txt


In [26]:
with open("all_chunks.txt", "r", encoding="utf-8") as f:
    all_chunks = [line.split(": ", 1)[1].strip() for line in f if line.startswith("Chunk")]

from langchain.docstore.document import Document

documents = [Document(page_content=chunk) for chunk in all_chunks]
print(f"Created {len(documents)} documents")

Created 9417 documents


In [39]:
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer

# Load chunks from all_chunks.txt
with open("all_chunks.txt", "r", encoding="utf-8") as f:
    all_chunks = [line.split(": ", 1)[1].strip() for line in f if line.startswith("Chunk")]

# Use a subset (e.g., 1,000) to avoid memory issues
documents = [Document(page_content=chunk) for chunk in all_chunks[:1000]]
print(f"Created {len(documents)} documents")

# Custom embedding class for SentenceTransformer
class SentenceTransformerEmbeddings:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True)[0].tolist()

# Initialize the custom embeddings
embeddings = SentenceTransformerEmbeddings()

# Create the vector store with the embeddings object
vector_store = FAISS.from_documents(documents, embeddings)
print("Vector store created")

# Perform the query using similarity_search_by_vector as a workaround
query = "What are the risks mentioned in Amazon's 10-K?"
query_embedding = embeddings.embed_query(query)  # Generate embedding for the query
results = vector_store.similarity_search_by_vector(query_embedding, k=5)
for i, doc in enumerate(results, 1):
    print(f"Result {i}: {doc.page_content[:200]}...")  # Print first 200 chars of each result

Created 1000 documents


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Vector store created
Result 1: o a number of risks,...
Result 2: Are Harmed by the Products We Sell or Manufacture...
Result 3: , and loss of critical data, and could prevent us from accepting and fulfilling customer...
Result 4: siness and Industry Risks...
Result 5: s. Any such disruptions or issues may harm our brand and business....


In [ ]:
# Import necessary libraries
import fitz  # PyMuPDF for PDF processing
import os
import time
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer

# Step 1: PDF Preprocessing and Chunking
pdf_files = [
    "amazon_10k_2024.pdf", "Tesla_10k_2024.pdf", "ExxonMobil_10k_2024.pdf",
    "apple_10k_2024.pdf", "Microsoft_10k_2024.pdf", "Alphabet_10k_2024.pdf",
    "JPMorgan_10k_2024.pdf", "Johnson&Johnson_10k_2024.pdf",
    "walmart_10k_2024.pdf", "Pepsico_10k_2024.pdf"
]

all_chunks = []
for pdf_file in pdf_files:
    print(f"Processing {pdf_file}...")
    try:
        doc = fitz.open(pdf_file)
        text = "".join(page.get_text() for page in doc)
        chunks = [text[i:i+500] for i in range(0, len(text), 500)]
        all_chunks.extend(chunks)
        print(f"Added {len(chunks)} chunks from {pdf_file}")
    except Exception as e:
        print(f"❌ Error processing {pdf_file}: {e}")

print(f"\n✅ Total chunks created: {len(all_chunks)}")
with open("all_chunks.txt", "w", encoding="utf-8") as f:
    for i, chunk in enumerate(all_chunks, 1):
        f.write(f"Chunk {i}: {chunk}\n")
print("📄 Chunks saved to all_chunks.txt")

# Step 2: Load Chunks
with open("all_chunks.txt", "r", encoding="utf-8") as f:
    all_chunks = [line.split(": ", 1)[1].strip() for line in f if line.startswith("Chunk")]

# ⚠️ TEMP: Reduce for testing - later increase to 5000 if memory allows
documents = [Document(page_content=chunk) for chunk in all_chunks[:2000]]
print(f"📄 Loaded {len(documents)} documents")

# Step 3: Custom Embedding Class (with optional GPU)
class SentenceTransformerEmbeddings:
    def __init__(self, model_name="all-MiniLM-L6-v2", use_gpu=False):
        device = "cuda" if use_gpu else "cpu"
        self.model = SentenceTransformer(model_name, device=device)

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True)[0].tolist()

# Step 4: Create or Load Vector Store
embeddings = SentenceTransformerEmbeddings(use_gpu=False)
vector_store_path = "faiss_store"

if os.path.exists(vector_store_path):
    vector_store = FAISS.load_local(vector_store_path, embeddings)
    print("📂 Loaded vector store from disk")
else:
    print("⏳ Embedding and building vector store...")
    start = time.time()
    vector_store = FAISS.from_documents(documents, embeddings)
    print(f"✅ Vector store created in {time.time() - start:.2f} seconds")
    vector_store.save_local(vector_store_path)
    print(f"💾 Vector store saved to '{vector_store_path}'")

# Step 5: CLI Query Interface
print("\n🧠 Enter queries below (type 'quit' to exit):")
while True:
    query = input("\n🔎 Your query: ")
    if query.lower() == 'quit':
        break
    query_embedding = embeddings.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=5)
    for i, doc in enumerate(results, 1):
        print(f"\nResult {i}:\n{doc.page_content[:300]}...")

# Step 6: Optional LangChain Agent Integration
try:
    from langchain.agents import initialize_agent, Tool

    tools = [
        Tool(
            name="Search10K",
            func=lambda q: [doc.page_content for doc in vector_store.similarity_search_by_vector(embeddings.embed_query(q), k=5)],
            description="Search across 10-K filings."
        )
    ]

    agent = initialize_agent(tools, embeddings, agent_type="zero-shot-react-description")
    print("\n🤖 Agent initialized. Try: `agent.run('Compare Microsoft and Tesla risks')`")
except ImportError:
    print("\n⚠️ LangChain agents not installed. Run: pip install -U langchain-agents")


Processing amazon_10k_2024.pdf...
Added 591 chunks from amazon_10k_2024.pdf
Processing Tesla_10k_2024.pdf...
Added 812 chunks from Tesla_10k_2024.pdf
Processing ExxonMobil_10k_2024.pdf...
Added 946 chunks from ExxonMobil_10k_2024.pdf
Processing apple_10k_2024.pdf...
Added 429 chunks from apple_10k_2024.pdf
Processing Microsoft_10k_2024.pdf...
Added 807 chunks from Microsoft_10k_2024.pdf
Processing Alphabet_10k_2024.pdf...
Added 742 chunks from Alphabet_10k_2024.pdf
Processing JPMorgan_10k_2024.pdf...
Added 2515 chunks from JPMorgan_10k_2024.pdf
Processing Johnson&Johnson_10k_2024.pdf...
Added 874 chunks from Johnson&Johnson_10k_2024.pdf
Processing walmart_10k_2024.pdf...
Added 775 chunks from walmart_10k_2024.pdf
Processing Pepsico_10k_2024.pdf...
Added 926 chunks from Pepsico_10k_2024.pdf

✅ Total chunks created: 9417
📄 Chunks saved to all_chunks.txt
📄 Loaded 2000 documents
⏳ Embedding and building vector store...


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


✅ Vector store created in 5.51 seconds
💾 Vector store saved to 'faiss_store'

🧠 Enter queries below (type 'quit' to exit):
